In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from moha.hamiltonians import HamHub
from RDMFS import MU_U2RDM
from utils import Vee
import inspect 
import scipy 
#from numba.openmp import openmp_context as openmp

In [2]:
N = 4
norbs = N
nelec = N
system = [(f"H{i}", f"H{i+1}", 1) for i in range(1, N)] + [(f"H{N}", f"H1", 1)]
print(system)
hubbard = HamHub(system,alpha=1, beta=2, u_onsite=np.ones((N)),
                      sym=1)

h_0 = hubbard.generate_zero_body_integral()
h_1 = hubbard.generate_one_body_integral(basis='spinorbital basis', dense=True)
h2 = hubbard.generate_two_body_integral(sym=8,basis='spinorbital basis',
                                                dense=True)

h2 = np.transpose(h2, (0, 2, 1, 3))
h2_hub_ab = h2[0:N,0:N,N:2*N,N:2*N]
h2_hub_aa = h2[0:N,0:N,0:N,0:N]
h2_hub_bb = h2[N:2*N,N:2*N,N:2*N,N:2*N]
h_1 = (h_1[0:N,0:N], h_1[N:2*N,N:2*N])
h2_hub = (h2_hub_aa, h2_hub_ab, h2_hub_bb)

cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e, ci = cisolver.kernel(h_1, h2_hub, N, N, ecore=h_0)

dm1, dm2  = cisolver.make_rdm12(ci, N, N)

print(dm1)

[('H1', 'H2', 1), ('H2', 'H3', 1), ('H3', 'H4', 1), ('H4', 'H1', 1)]
[[ 1.00000000e+00 -4.97169175e-01 -2.25072171e-17 -4.97169175e-01]
 [-4.97169175e-01  1.00000000e+00 -4.97169175e-01  2.44027023e-17]
 [-2.25072171e-17 -4.97169175e-01  1.00000000e+00 -4.97169175e-01]
 [-4.97169175e-01  2.44027023e-17 -4.97169175e-01  1.00000000e+00]]
